In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('C:/Users/Thiago/Desktop/Potifolio/Projeto calculadora de imoveis/Data Sets/todos_os_resultados.csv')

In [3]:
sum(df.duplicated())

10

In [4]:
df = df.drop_duplicates(keep="first", subset=[coluna for coluna in df.columns if coluna!="crawled_at"]).reset_index(drop=True)

In [5]:
df.shape

(6946, 13)

In [6]:
filtro_de_anuncios = [str(_id).isnumeric() for _id in df["id"]]

In [7]:
df = df[filtro_de_anuncios].reset_index(drop=True)

In [8]:
df.shape

(6946, 13)

In [9]:
set(df["rooms"])

{'1 Quarto',
 '2 Quartos',
 '3 Quartos',
 '4 Quartos',
 '5 Quartos',
 '6 Quartos',
 '7 Quartos',
 '9 Quartos'}

In [10]:
set(df["rooms"].str.split(" ").str[0].str.replace("--" , "0"))

{'1', '2', '3', '4', '5', '6', '7', '9'}

In [11]:
df["Quarto"] = (df["rooms"]
                     .str.split(" ")
                     .str[0]
                     .str.replace("--","0")
                     .astype(int))

In [12]:
df["Banheiro"] = (df["bathrooms"]
                         .str.split(" ")
                         .str[0]
                         .str.replace("--","0")
                         .astype(int))

In [13]:
df["Garagem"] = (df["garages"]
                       .str.split(" ")
                       .str[0]
                       .str.replace("--","0")
                       .astype(int))

In [14]:
df["condo"] = df["condo"].fillna("MISSING")

In [15]:
df["condominio"] = [int(w.split("R$ ")[1].replace(".","")) if w!="MISSING" else np.nan for w in df["condo"]]

In [16]:
#df["price_limpo"] = [int(w.split("R$ ")[1].split(" /mês")[0].split(" /dia")[0].replace(".","")) for w in df["price"]]
df["preço"] = [int(w.split("R$ ")[1].split(" /")[0].replace(".","")) for w in df["price"]]

In [17]:
df["area_limpo"] = df["area"].astype(int)

In [18]:
df["crawled_at"] = pd.to_datetime(df["crawled_at"], format="%Y-%m-%d %H:%M")

In [19]:
df = df.drop(columns = ["area", "rooms", "bathrooms", "garages", "price", "condo"])

In [20]:
df["bairro"] = (df["address"]
                      .str.split("- ")
                      .str[1]
                      .str.split(",")
                      .str[0])

In [21]:
df.loc[df["bairro"].isin(["RJ"]), "bairro"] = (df
                                                     .loc[df["bairro"].isin(["RJ"]), "address"]
                                                     .str.split(",").str[0])

In [22]:
df.loc[df["crawler"]!=df["bairro"],"bairro"].unique()

array(['Méier', 'Recreio Dos Bandeirantes', 'Barra da Tijuca'],
      dtype=object)

In [23]:
df["crawler"] = (df["crawler"]
                 .str.lower()
                 .str.replace(" ","_"))

In [24]:
df["bairro"] = (df["bairro"]
                      .str.lower()
                      .str.replace(" ","_")
                      .str.normalize("NFKD")
                      .str.encode("ascii", errors="ignore")
                      .str.decode("utf-8"))

In [25]:
df.loc[df["crawler"]!=df["bairro"], ["crawler","bairro"]]

,crawler,bairro
5437,recreio,recreio_dos_bandeirantes
5438,recreio,recreio_dos_bandeirantes
5439,recreio,recreio_dos_bandeirantes
5440,recreio,recreio_dos_bandeirantes
5441,recreio,recreio_dos_bandeirantes
...,...,...
6941,barra,barra_da_tijuca
6942,barra,barra_da_tijuca
6943,barra,barra_da_tijuca
6944,barra,barra_da_tijuca


In [26]:
df = df.drop(columns="bairro")

In [27]:
df["amenities"]

0       Elevador\nPlayground\nAceita animais\nSalão de...
1        Piscina\nVaranda\nAcademia\nElevador\nPlayground
2       Varanda\nAcademia\nInterfone\nSalão de jogos\n...
3       Churrasqueira\nElevador\nCondomínio fechado\nP...
4       Piscina\nChurrasqueira\nElevador\nAcademia\nVa...
                              ...                        
6941                                                  NaN
6942                                                  NaN
6943                                                  NaN
6944                                                  NaN
6945    Piscina\nAcademia\nChurrasqueira\nElevador\nPl...
Name: amenities, Length: 6946, dtype: object

In [28]:
df["amenities"] = df["amenities"].str.split("\n")

In [29]:
df["amenities"] = df["amenities"].fillna("")

In [30]:
amenities = []
for am in df["amenities"]:
    if am != "":
        amenities = amenities + am

In [31]:
unico_amenities = list(set(amenities))

In [32]:
df["amenities"] = df["amenities"].str.join(", ")

In [33]:
for amenity in unico_amenities:
    print(amenity)
    df[amenity]=0
    df.loc[df["amenities"].str.contains(amenity), amenity] = 1

Vista exterior
Cabeamento estruturado
Circuito de segurança
Quadra poliesportiva
Espaço gourmet
Portaria 24h
Porcelanato
Ar-condicionado
Varanda
Acesso para deficientes
Piscina infantil
Gás Encanado
Fogão
Cozinha grande
Sala grande
Quintal
Salão de jogos
Janela grande
Isolamento térmico
Conexão à internet
Sala de jantar
Vigia
Copa
Janela de alumínio
Mobiliado
Spa
Bicicletário
Próximo a transporte público
Vista para lago
Ambientes integrados
Próximo a hospitais
Portão eletrônico
Sala de massagem
Mais de um andar
Armário embutido no quarto
Piso laminado
Espaço teen
Condomínio fechado
Interfone
Área de serviço
Cozinha
Churrasqueira
Perto de vias de acesso
Móvel planejado
Hall de entrada
Espaço verde / Parque
Vista para a montanha
Área de lazer
Escritório
Salão de festas
Quadra de tênis
Churrasqueira na varanda
Jardim
Depósito
Cozinha americana
Vista para o mar
Piscina para adulto
Recepção
Elevador
Próximo a shopping
Piso de madeira
Sauna
Andar inteiro
Armário na cozinha
Pista de cooper
Pi

In [36]:
df.to_csv("C:/Users/Thiago/Desktop/Potifolio/Projeto calculadora de imoveis/Data Sets/aptos_csv.csv", index=False)